# DynoRRT

DynoRRT is a library for Sample-based Motion Planning (e.g., Rapidly Exploring Random Trees (RRT), 
Probabilistic Roadmaps (PRM)...)

Did you know you can get state-of-the-art perfomance with just pip install (and yes, it will be much faster than OMPL). No ROS, no system-wide packages, no MOVEIT, no OMPL.

You can install DynoRRT in 5 seconds, write a problem in 60 seconds and solve it in milliseconds. 

You can define you own collision function or define your problems using URDF Files. Pinocchio and HPP-FCL are used for computing collisions. They are linked statically, so you don't need them at runtime -- or it's fine if you have another version of the libraries.

The Python package provides a couple of utilities to visualize the problems using Pinocchio and Meshcat, but you can use the viewer you want.

The library is in alpha state. We are targetting a public release 0.1 in January.  C++ packaging is still in development. Feel free to open a github issue or pull request!

In [ ]:
# Install -- Uncomment if you don't have yet pydynorrt 
# 
# !pip3 install pydynorrt

In [ ]:
import numpy as np
import pydynorrt as pyrrt
import time

In [ ]:
# Define a planning problem

# Start and Goal
start = np.array([-0.62831853, 0.0, 0.0, 0.0, 0.9424778, 0.0, -0.9424778])
goal = np.array([0.62831853, 0.2, 0.3, 0.0, 0.9424778, 0.0, -0.9424778])

# State Bounds
lb = np.array([-1, -1, -1, -1.5708, -1.5708, -1.5708, -1.5708])
ub = np.array([1, 1, 1, 1.5708, 1.5708, 1.5708, 1.5708])

# We need a robot and an environment. You can define 
# everything in a single URDF (currently supported) or in 
# two URDFS (will be supported soon)
urdf = pyrrt.DATADIR + "models/point_payload_two_robots.urdf"

# Indicate which contact pairs should be ignore (e.g., usually consecutive links)
srdf = pyrrt.DATADIR + "models/point_payload_two_robots.srdf"



In [ ]:
# Collision checking using the URDF model. 
# Adding your custom Collision checking in python is also possible! (see below)
# Comming Soon: parallel collision checking. 
cm = pyrrt.Collision_manager_pinocchio()
cm.set_urdf_filename(urdf)
cm.set_srdf_filename(srdf)
cm.build()

# We can double check that start and goal are collision free!
tic = time.time()
assert cm.is_collision_free(start)
assert cm.is_collision_free(goal)
mid = (goal + start) / 2.0
assert not cm.is_collision_free(mid)

# Collision Checking is fast -- Thanks Pinocchio and Hpp-fcl
print("Average Time of 1 collision in C++ [ms]", cm.get_time_ms()/3.)

In [ ]:
# Let's visualize the problem
# You can use your favorite library to visualize a URDF. 
# In this case, I am using Pinocchio and Meshcat. 

# They are great. I strongly recommend to check 
# all the examples and tests in Pinocchio source code 
# and the tutorial in https://github.com/Gepetto/supaero2022

# Note that the Pinocchio package that we import know is
# not responsible for collision checking!
#!pip3 install meshcat
#!pip3 install pin

In [ ]:
import meshcat
import pinocchio
from pydynorrt import pin_more # small utility functions to visualize motion planning 
# problems.

viewer = meshcat.Visualizer()
viewer_helper = pin_more.ViewerHelperRRT(viewer, urdf, srdf, start, goal)

In [ ]:
# Create the Planner.
# RRT_X is and RRT planner for the space Rn, where n is given at runtime. 
# E.g., RRT_R2() (TODO) knows the state space at compile time, 
# which means faster memory allocation and nearest neighbour queries 
# (but note that runtime is often dominated by colliision checking)

# For state space, we currently support any combination and composition of
# of SO(3), Rn, SO(2) and time, 

rrt = pyrrt.RRT_X()
rrt.set_start(start)
rrt.set_goal(goal)
rrt.init(7)
rrt.set_is_collision_free_fun_from_manager(cm)
rrt.set_bounds_to_state(lb, ub)

# Options are provide through config files/string in TOML format. 
# If an option is not present, we use the default value. 
config_string ="""
[RRT_options]
max_it = 20000
max_num_configs = 20000
max_step = 1.0 
goal_tolerance = 0.001
collision_resolution = 0.2
goal_bias = 0.1
store_all = false
"""
# rrt.read_cfg_file(my_config_file) # You can also 
# provde the name of a config file. 
rrt.read_cfg_string(config_string)

# lets plan -- it is fast. 
out = rrt.plan()
parents = rrt.get_parents()
configs = rrt.get_configs()
path = rrt.get_path()
fine_path = rrt.get_fine_path(.01)

# Some Planners will output a lot of information useful for visualization 
# and for debugging -- see below. You can get this directly in a dictionary. 
# (path, parents, configs) are also available as planner data.
planner_data = {}
rrt.get_planner_data(planner_data)

In [ ]:
# Now let's visualize the solution and the search of the planer. 
# The system is 7D -- To visualize the Search tree, we plot only 
# the position of the center ball. 
robot = viewer_helper.robot
viz = viewer_helper.viz

idx_vis_name = "point_mass"
IDX_VIS = robot.model.getFrameId(idx_vis_name)
display_count = 0 # just to enumerate the number 
# of objects we create in the visualizer.
for i, p in enumerate(parents):
    if p != -1:
        q1 = configs[i]
        q2 = configs[p]
            # pin_more 
        pin_more.display_edge(robot, q1, q2, IDX_VIS, display_count, viz, radius=0.005,
                                  color=[0.2, 0.8, 0.2, .9])
        display_count += 1

for i in range(len(path) - 1):
    q1 = path[i]
    q2 = path[i + 1]
    pin_more.display_edge(robot, q1, q2, IDX_VIS, display_count, viz, radius=0.02,
                                  color=[0.0, 0.0, 1., 0.5])
    display_count += 1

# Finally, we can visualize the path of the robot :)
for p in fine_path:
    viz.display(np.array(p))
    time.sleep(0.01)


In [ ]:
# You have finished the first tutorial of DynoRRT. 
# Now we will showcase a couple of more robotics problems, 
# planners and options!

In [ ]:
# First, let's solve another problem. 
# We want to get a path inside the path. 

In [ ]:
# Let's Define a second planning problem, 
# where a robot manipulator (ur5) has find a path 
# to get inside a bin/shelf.


lb = np.array([
    -3.14,
    -3.14,
    -3.14,
    -3.14,
    -3.14,
    -3.14
  ])
ub = np.array([
    3.14,
    3.14,
    3.14,
    3.14,
    3.14,
    3.14
  ])
start = np.array([
    0,
    -1.5,
    2.1,
    -0.5,
    -0.5,
    0
  ])

goal =  np.array([
    3.1,
    -1.0,
    1,
    -0.5,
    -0.5,
    0
  ])


# We need a robot and an environment. You can define 
# everything in a single URDF (currently supported) or in 
# two URDFS (will be supported soon)
urdf = pyrrt.DATADIR + "models/ur5_robot_with_box.urdf"

# Indicate which contact pairs should be ignore (e.g., usually consecutive links)
srdf = pyrrt.DATADIR + "models/ur5_with_box.srdf"


In [ ]:
# Collision checking using the URDF model. 
# Note: we use simplified collision shapes for collision 
# checking, instead of the original visual mesh.

cm = pyrrt.Collision_manager_pinocchio()
cm.set_urdf_filename(urdf)
cm.set_srdf_filename(srdf)
cm.build()

# We can double check that start and goal are collision free!
tic = time.time()
assert cm.is_collision_free(start)
assert cm.is_collision_free(goal)

# Collision Checking is fast -- Thanks Pinocchio and Hpp-fcl
print("Average Time of 1 collision in C++ [ms]", cm.get_time_ms()/3.)

In [ ]:
# We are creating another viewer (note that the URL may be different!)
# Collision shapes used for planning are shown with a translucid color. 
viewer = meshcat.Visualizer()
viewer_helper = pin_more.ViewerHelperRRT(viewer, urdf, srdf, start, goal)

In [ ]:
# We have implemented the most commonly used algorithm for 
# sample based motion planning: RRT, RRT*, RRTConnect, (lazy)PRM(*)...

# RRT_Connect is often the fastest planner (specially for manipulation planning) but
# the paths might be far from optimal. 
rrt = pyrrt.RRTConnect_X()
rrt.set_start(start)
rrt.set_goal(goal)
rrt.init(6)

config_str = """
[RRTConnect_options]
max_it = 100000
collision_resolution = 0.2
max_step = 1.0
max_num_configs = 100000
"""
rrt.read_cfg_string(config_str)
rrt.set_is_collision_free_fun_from_manager(cm)
rrt.set_bounds_to_state(lb, ub)

# lets plan -- it is fast. 
out = rrt.plan()
path = rrt.get_path()
fine_path = rrt.get_fine_path(.01)
planner_data = rrt.get_planner_data()
print("planner_data")
for i in planner_data:
    print(i)
parents_backward = planner_data["parents_backward"]
configs_backward = [ np.array(x) for x in planner_data["configs_backward"]]
parents = planner_data["parents"]
configs = [np.array(x) for x in planner_data["configs"]]


In [ ]:
# planner_data is a python dictionary 
for i in planner_data:
    print(i)

In [ ]:
# Lets visualize the results.
# Now let's visualize the solution and the search of the planer. 
# The system is 7D -- To visualize the Search tree, we plot only 
# the position of the center ball. 
robot = viewer_helper.robot
viz = viewer_helper.viz

idx_vis_name = "tool0"
IDX_VIS = robot.model.getFrameId(idx_vis_name)
display_count = 0 # just to enumerate the number 

# We display the forward and backward tree in two different colors
for i, p in enumerate(parents):
    if p != -1:
        q1 = configs[i]
        q2 = configs[p]
            # pin_more 
        pin_more.display_edge(robot, q1, q2, IDX_VIS, display_count, viz, radius=0.005,
                                  color=[0.2, 0.8, 0.2, .9])
        display_count += 1

for i, p in enumerate(parents_backward):
    if p != -1:
        q1 = configs_backward[i]
        q2 = configs_backward[p]
            # pin_more 
        pin_more.display_edge(robot, q1, q2, IDX_VIS, display_count, viz, radius=0.005,
                                  color=[0.8, 0.2, 0.2, .9])
        display_count += 1
# 

for i in range(len(path) - 1):
    q1 = path[i]
    q2 = path[i + 1]
    pin_more.display_edge(robot, q1, q2, IDX_VIS, display_count, viz, radius=0.02,
                                  color=[0.0, 0.0, 1., 0.5])
    display_count += 1

# Finally, we can visualize the path of the robot :)
for p in fine_path:
    viz.display(np.array(p))
    time.sleep(0.01)

In [ ]:
# Coming soon -- two more examples in robotics! 